In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta

from skimage.metrics import structural_similarity
import numpy as np


%matplotlib inline


In [3]:
def loadImage(imgNameFull: str) -> dict:
    if imgNameFull[-3:] != "jpg" and imgNameFull[-3:]!= "png":
        return
    [camName, dt]=imgNameFull[:-4].split(sep = "_")
    img = cv2.imread(imgNameFull)
    return {"cameraName": camName, "dateTime": dt, "frame": img}


In [4]:
fileLocation = "D:\\shitcam"
dir_list = os.listdir(fileLocation)
dir_list = [fileLocation + "\\"+ a for a in dir_list]  
print("Files and directories in '", fileLocation, "' :") 

# print the list
print(len(dir_list), " files")


Files and directories in ' D:\shitcam ' :
855  files


In [5]:
def cvtDateTime(dt:str) :
    dt_list = [int(dt[a:a+2]) for a in range(0,len(dt),2)]
    dt_list[0] = dt_list[0]+2000
    #print(dt_list)
    return datetime(dt_list[0],dt_list[1],dt_list[2],dt_list[3],dt_list[4], dt_list[5])


def checkvalid(prev:dict, now:dict) ->bool:
    if prev["cameraName"]!=now["cameraName"]:
        return False
    if (cvtDateTime(now["dateTime"]) - cvtDateTime(prev["dateTime"])).total_seconds()>60:
        return False
    return True


In [6]:
## Test out the cv2 image background subtraction feature for each pair of image
prev = loadImage(dir_list[0])

validPair = []
for i in range(1,len(dir_list)):
    print(dir_list[i])
    if dir_list[i][-3:] != "jpg" and dir_list[i][-3:]!= "png":
        continue

    now = loadImage(dir_list[i])
    
    if not checkvalid(prev,now):
        continue
    
    validPair.append((prev,now))

    prev = now
    
    


D:\shitcam\LivingRoomCamera_230206132243.jpg
D:\shitcam\LivingRoomCamera_230206132303.jpg
D:\shitcam\LivingRoomCamera_230206132323.jpg
D:\shitcam\LivingRoomCamera_230206132343.jpg
D:\shitcam\LivingRoomCamera_230206132403.jpg
D:\shitcam\LivingRoomCamera_230206132423.jpg
D:\shitcam\LivingRoomCamera_230206132443.jpg
D:\shitcam\LivingRoomCamera_230206132503.jpg
D:\shitcam\LivingRoomCamera_230206132523.jpg
D:\shitcam\LivingRoomCamera_230206132543.jpg
D:\shitcam\LivingRoomCamera_230206132603.jpg
D:\shitcam\LivingRoomCamera_230206132623.jpg
D:\shitcam\LivingRoomCamera_230206132643.jpg
D:\shitcam\LivingRoomCamera_230206132704.jpg
D:\shitcam\LivingRoomCamera_230206132724.jpg
D:\shitcam\LivingRoomCamera_230206132744.jpg
D:\shitcam\LivingRoomCamera_230206132804.jpg
D:\shitcam\LivingRoomCamera_230206132824.jpg
D:\shitcam\LivingRoomCamera_230206132844.jpg
D:\shitcam\LivingRoomCamera_230206132904.jpg
D:\shitcam\LivingRoomCamera_230206132924.jpg
D:\shitcam\LivingRoomCamera_230206132944.jpg
D:\shitcam

In [9]:
def calculateSSIMIndex(before,after):
    
    # Convert images to grayscale
    before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
    after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)
    
    # Compute SSIM between the two images
    (score, diff) = structural_similarity(before_gray, after_gray, full=True)
    print("Image Similarity: {:.4f}%".format(score * 100))

    # The diff image contains the actual image differences between the two images
    # and is represented as a floating point data type in the range [0,1] 
    # so we must convert the array to 8-bit unsigned integers in the range
    # [0,255] before we can use it with OpenCV
    diff = (diff * 255).astype("uint8")
    diff_box = cv2.merge([diff, diff, diff])

    # Threshold the difference image, followed by finding contours to
    # obtain the regions of the two input images that differ
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    mask = np.zeros(before.shape, dtype='uint8')
    filled_after = after.copy()

    for c in contours:
        area = cv2.contourArea(c)
        if area > 40:
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
            cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
            cv2.rectangle(diff_box, (x, y), (x + w, y + h), (36,255,12), 2)
            cv2.drawContours(mask, [c], 0, (255,255,255), -1)
            cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

    cv2.imshow('before', before)
    cv2.imshow('after', after)
    #cv2.imshow('diff', diff)
    #cv2.imshow('diff_box', diff_box)
    #cv2.imshow('mask', mask)
    #cv2.imshow('filled after', filled_after)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [10]:
for pair in validPair[0:50]:
    print(pair[0]["dateTime"],", ", pair[1]["dateTime"])
    calculateSSIMIndex(pair[0]["frame"], pair[1]["frame"])




230206132223 ,  230206132243
Image Similarity: 80.4279%
230206132243 ,  230206132303
Image Similarity: 66.3572%
230206132303 ,  230206132323
Image Similarity: 65.1644%
230206132323 ,  230206132343
Image Similarity: 68.2610%
230206132343 ,  230206132403
Image Similarity: 65.2434%
230206132403 ,  230206132423
Image Similarity: 63.3226%
230206132423 ,  230206132443
Image Similarity: 64.8384%
230206132443 ,  230206132503
Image Similarity: 69.3186%
230206132503 ,  230206132523
Image Similarity: 68.2893%
230206132523 ,  230206132543
Image Similarity: 75.0026%
230206132543 ,  230206132603
Image Similarity: 74.8839%
230206132603 ,  230206132623
Image Similarity: 71.9956%
230206132623 ,  230206132643
Image Similarity: 72.0120%
230206132643 ,  230206132704
Image Similarity: 73.9004%
230206132704 ,  230206132724
Image Similarity: 73.3500%
230206132724 ,  230206132744
Image Similarity: 72.3243%
230206132744 ,  230206132804
Image Similarity: 72.1123%
230206132804 ,  230206132824
Image Similarity: 7